### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
from keras.utils import to_categorical

Using TensorFlow backend.


### Load Raw Training Data

In [2]:
inpData = []
labels = []
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
        

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
index = 0
while index + 1 < len(inpData):
    if inpData[index + 1] > maxThreshold:
        target.append(1)
    elif inpData[index + 1] < minThreshold:
        target.append(-1)
    else:
        target.append(0)
    index += 1

In [ ]:
print(target)

### Preprocessing Input Data

In [4]:
cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1
    

In [5]:
cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

In [6]:
#len(inpData)
X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.25, random_state=42)

In [ ]:
cpuUsageData.shape
#np.shape(cpuUsage)

In [ ]:
cpuScalingCategorical.shape

In [7]:
model = Sequential()
#model.add(SpatialDropout1D(0.3))
model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,50,1), return_sequences = True))
model.add(Dense(50, activation = 'tanh'))
#model.add(SpatialDropout1D(0.3))
model.add(LSTM(25,return_sequences = False))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
dense_1 (Dense)              (None, 50, 50)            2550      
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 78        
Total params: 20,628
Trainable params: 20,628
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs = 50,validation_data = [X_test, y_test], callbacks = callbacks_list,verbose=0)

In [ ]:
model.save

In [ ]:
res = model.predict(X_test,batch_size = 1)

In [ ]:
print(res)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])